Setup

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import layers
import dask.dataframe as dd

2024-02-16 16:58:14.490498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 16:58:14.493586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 16:58:14.799629: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 16:58:16.050983: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-16 16:58:56.689656: W tensorflow/compiler/tf2

In [3]:
# Read dataset as dask dataframe

root = "/lustre/home/debnathk/dleps/code/DLEPS/reference_drug/"
# ssp_train_dask = dd.read_hdf(root + 'ssp_data_train.h5')

import h5py
import dask.array as da
from dask import delayed

# Define a delayed function to read a dataset from the HDF5 file
@delayed
def read_hdf5(path, dataset):
    with h5py.File(path, 'r') as f:
        return f[dataset][:]

# Use the delayed function to read the dataset
ssp_train_dask = read_hdf5(root + 'ssp_data_train.h5', 'data')
ssp_test_dask = read_hdf5(root + 'ssp_data_test.h5', 'data')

# Convert the delayed object to a Dask array
data = da.from_delayed(ssp_train_dask, shape=(44988, 207, 3072), dtype=float)

In [4]:
data

dask.array<from-value, shape=(44988, 207, 3072), dtype=float64, chunksize=(44988, 207, 3072), chunktype=numpy.ndarray>

In [5]:
# Convert Dask arrays to TensorFlow datasets for training
x_train = tf.data.Dataset.from_tensor_slices(data).batch(32)
# x_test = tf.data.Dataset.from_tensor_slices(ssp_train_dask).batch(32)

2024-02-16 17:04:47.538859: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 228864393216 exceeds 10% of free system memory.


In [6]:
np.array(ssp_train_dask.sample(fraction=0.1))

# Use TFRecord

array(Delayed('sample-f89b370c-dea0-400c-a7df-706de834280b'), dtype=object)

In [7]:
data.sample(fraction=0.1)

AttributeError: 'Array' object has no attribute 'sample'

In [ ]:
# Load dataset
# root = "/lustre/home/debnathk/dleps/code/DLEPS/reference_drug/"
# import h5py

# h5f = h5py.File(root + 'ssp_data_train.h5', 'r')
# ssp_train = h5f['data'][:]
# h5f = h5py.File(root + 'ssp_data_test.h5', 'r')
# ssp_test = h5f['data'][:]

# print(ssp_train.shape)
# print(ssp_test.shape)

In [ ]:
# plt.hist(ssp_train[:, 100, :].sum(axis=1) / 4000)


Create a sampling layer

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

Build the encoder

In [ ]:
latent_dim = 56

encoder_inputs = keras.Input(shape=(207, 3072))
# Add dense layer
x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 207, 3072)]          0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 104, 32)              294944    ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 52, 64)               6208      ['conv1d[0][0]']              
                                                                                                  
 flatten (Flatten)           (None, 3328)                 0         ['conv1d_1[0][0]']            
                                                                                            

Build the decoder

In [ ]:
# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16, activation="relu")(latent_inputs)
x = layers.Reshape((1, 16))(x)
x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
decoder_outputs = layers.Dense(207 * 3072, activation="sigmoid")(x)
decoder_outputs = layers.Reshape((207, 3072))(decoder_outputs)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 56)]              0         
                                                                 
 dense_1 (Dense)             (None, 16)                912       
                                                                 
 reshape (Reshape)           (None, 1, 16)             0         
                                                                 
 conv1d_transpose (Conv1DTr  (None, 2, 64)             3136      
 anspose)                                                        
                                                                 
 conv1d_transpose_1 (Conv1D  (None, 4, 32)             6176      
 Transpose)                                                      
                                                                 
 flatten_1 (Flatten)         (None, 128)               0   

Define the VAE as a Model with a custom train_step

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(data, reconstruction))
            kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Instantiate the VAE model
vae = VAE(encoder=encoder, decoder=decoder)

# Compile the model
vae.compile(optimizer=tf.keras.optimizers.Adam())
num_epochs, batch_size = 3, 32


In [ ]:
# Generate batches of data from our dask dataframe
def dask_data_generator(df, fraction=0.1):
    while True:
        batch = df.sample(fraction=fraction)
        X = batch.iloc[:, :]
        yield X.compute()

In [ ]:
dask_data_generator(ssp_train_dask)

<generator object dask_data_generator at 0x7f1a15bd0d60>

In [ ]:
import matplotlib.pyplot as plt

# Assuming you've already trained the VAE using the code you provided

# Train the VAE model
history = vae.fit_generator(generator= dask_data_generator(ssp_train_dask),
                            epochs=num_epochs,
                            steps_per_epoch=100,
                            validation_data=dask_data_generator(ssp_test_dask))

# Plot the losses over epochs
def plot_losses(history):
    loss = history.history['loss']
    reconstruction_loss = history.history['reconstruction_loss']
    kl_loss = history.history['kl_loss']

    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, loss, label='Total Loss')
    plt.plot(epochs, reconstruction_loss, label='Reconstruction Loss')
    plt.plot(epochs, kl_loss, label='KL Divergence Loss')

    plt.title('Training Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(root + 'training_losses.png')
    plt.show()

# Plot the losses
plot_losses(history)

/tmp/ipykernel_3168275/1319909898.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = vae.fit_generator(generator= dask_data_generator(ssp_train_dask),


AttributeError: 'numpy.ndarray' object has no attribute 'sample'

In [ ]:
import keras
import tensorflow

keras.__version__
tensorflow.__version__

'2.15.0'